# Simple Data Source Q&A

This is a sample implementation of an analytical Agent for Tableau built with Langgraph

### Dependencies and Environment Variables

First we begin with the dependencies and environment variables needed to run the Agent graph

In [2]:
import os
from dotenv import load_dotenv
from IPython.display import display, Markdown

# base langchain library imports
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

# community contributions
from community.langchain_community.tools.tableau.simple_datasource_qa import initialize_simple_datasource_qa
# sample tableau tools
from agents.tools import tableau_metrics, tableau_workbooks, tavily_tool


# environment variables available to current process and sub processes
load_dotenv()
# variables for authenticating and interacting with a Tableau site
tableau_domain = os.environ['TABLEAU_DOMAIN']
tableau_site = os.environ['TABLEAU_SITE']
tableau_jwt_client_id = os.environ['TABLEAU_JWT_CLIENT_ID']
tableau_jwt_secret_id = os.environ['TABLEAU_JWT_SECRET_ID']
tableau_jwt_secret = os.environ['TABLEAU_JWT_SECRET']
tableau_api_version = os.environ['TABLEAU_API_VERSION']
tableau_user = os.environ['TABLEAU_USER']
# the target data source for this Tool
datasource_luid = os.environ['DATASOURCE_LUID']
# variables to control LLM models for the Agent and Tools
open_api_key = os.environ["OPENAI_API_KEY"]
agent_llm_model = os.environ['AGENT_MODEL']
tooling_llm_model = os.environ['TOOLING_MODEL']

### Agent Graph

Then we define the Agent graph using Langgraph's prebuilt `create_react_agent` combined with the a tool to query a Tableau data source for conversational analytics

In [3]:
# configure running model for the agent
llm = ChatOpenAI(
    model=agent_llm_model,
    api_key=open_api_key,
    temperature=0,
    verbose=True,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

# Tableau VizQL Data Service Query Tool
tableau_datasource = initialize_simple_datasource_qa(
    domain=tableau_domain,
    site=tableau_site,
    jwt_client_id=tableau_jwt_client_id,
    jwt_secret_id=tableau_jwt_secret_id,
    jwt_secret=tableau_jwt_secret,
    tableau_api_version=tableau_api_version,
    tableau_user=tableau_user,
    datasource_luid=datasource_luid,
    tooling_llm_model=tooling_llm_model
)

#  
web_search = tavily_tool()

# load the List of Tools to be used by the Agent
tools = [ tableau_datasource, tableau_metrics, tableau_workbooks ]

# set agent debugging state
if os.getenv('DEBUG') == '1':
    debugging = True
else:
    debugging = False

# define the agent graph
query_agent = create_react_agent(
    model=llm,
    tools=tools,
    debug=debugging
)

### Interacting with the Analytical Agent

After constructing an Analytical Agent with the power to query a Tableau data source, we proceed to ask questions, set tasks and issue commands to the Agent

In [5]:
# question or task sent to the agent
message_string0 = 'give me an update on my kpis'
message_string1 = 'show me total sales, average discount, profits by region sorted by profit descending'
message_string2 = 'what dashboards cover the subject of shipping?'
message_string3 = 'explain row-level security for Tableau using their offical web docs'


# Run the agent
messages = query_agent.invoke({"messages": [("human", message_string3)]})
agent_message = "\n\nAnalytics Agent:\n\n" + messages['messages'][3].content
display(Markdown(agent_message))



Analytics Agent:

Row-level security (RLS) in Tableau is a method used to restrict access to specific rows of data within a workbook based on the user viewing it. This is different from Tableau permissions, which manage access to content and feature functionalities. For instance, while permissions determine whether a user can comment on or edit a workbook, RLS ensures that two users viewing the same dashboard will only see the data they are authorized to access.

Tableau provides several approaches to implement row-level security:

1. **User Filters**: This method involves creating user filters and manually mapping users to specific values. While this approach is straightforward, it can be high maintenance and may lead to security vulnerabilities if not managed properly.

For more detailed information, you can refer to the official Tableau documentation on [Row-Level Security Best Practices](https://help.tableau.com/current/server/en-us/rls_bestpractices.htm) and [User Filters](https://help.tableau.com/current/pro/desktop/en-us/publish_userfilters.htm).